In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y = train['label']

X = train.drop('label', axis=1)

In [6]:
X.shape

(42000, 784)

In [7]:
X = X.values.reshape(-1, 28, 28, 1)

In [8]:
X.shape

(42000, 28, 28, 1)

In [9]:
# Scaling
X = X / 255

In [10]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y)

In [11]:
y[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=26)

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

In [14]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D())
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D())
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Train on 37800 samples, validate on 4200 samples
Epoch 1/30
37800/37800 [==============================] - 10s 272us/sample - loss: 0.3141 - accuracy: 0.9015 - val_loss: 0.0792 - val_accuracy: 0.9786
Epoch 2/30
37800/37800 [==============================] - 6s 162us/sample - loss: 0.1033 - accuracy: 0.9711 - val_loss: 0.0511 - val_accuracy: 0.9843
Epoch 3/30
37800/37800 [==============================] - 6s 167us/sample - loss: 0.0774 - accuracy: 0.9779 - val_loss: 0.0570 - val_accuracy: 0.9821
Epoch 4/30
37800/37800 [==============================] - 6s 159us/sample - loss: 0.0636 - accuracy: 0.9817 - val_loss: 0.0383 - val_accuracy: 0.9881
Epoch 5/30
37800/37800 [==============================] - 7s 183us/sample - loss: 0.0547 - accuracy: 0.9845 - val_loss: 0.0364 - val_accuracy: 0.9874
Epoch 6/30
37800/37800 [==============================] - 6s 166us/sample - loss: 0.0502 - accuracy: 0.9857 - val_loss: 0.0410 - val_accuracy: 0.9855
Epoch 7/30
37800/37800 [==========================

In [16]:
test.shape

(28000, 784)

In [17]:
test = test.values.reshape(-1, 28, 28, 1)

In [18]:
test = test / 255

In [19]:
results = model.predict(test)

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [20]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)